# Firebase

In [15]:
import os

from dotenv import load_dotenv
from pymongo import MongoClient
from pymongo.server_api import ServerApi

load_dotenv()

mongo_uri = os.environ.get("MONGO_URL")
mongo_certificate = os.environ.get("MONGO_CERTIFICATE")
client = MongoClient(mongo_uri,
                     tls=True,
                     tlsCertificateKeyFile=mongo_certificate,
                     server_api=ServerApi('1'))
db = client['spotifai']
track_collection = db['tracks']

limit = 1000
offset = 0
completed = False

track_data = []
query = {
    "$and": [
        {
            "tags": {
                "$ne": {
                    "$regex": "^-?[0-9]+$",
                    "$options": "m"
                }
            }
        },
        {
            "tags": {
                "$ne": "[]",
            }
        }
    ]
}
docs = track_collection.find(query).skip(offset).limit(limit)
while not completed:
    pass
    docs = track_collection.find(query).skip(offset).limit(limit)
    current_track_data = [doc for doc in docs]
    completed = len(current_track_data) == 0
    track_data += current_track_data
    offset += limit


In [12]:
#track_collection.ensureIndex({"id": 1}, {"unique": True, "dropDups": True})
#track_collection.create_index("id")
def func(doc):
    doc.dups.shift()
    db.events.remove({ "id": {"$in": doc.dups }})

track_collection.aggregate([
    { "$group": {
        "_id": { "id": "%id" },
        "dups": { "$push": "$_id" },
        "count": { "$sum": 1 }
    }},
    { "$match": { "count": { "$gt": 1 } }}
]).forEach(func)

track_collection.createIndex({"id":1 },{"unique":True})

ServerSelectionTimeoutError: No replica set members match selector "Primary()", Timeout: 30s, Topology Description: <TopologyDescription id: 63c566d408aa522eb31a0017, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('ac-i0nvasr-shard-00-00.ikgqzkz.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.022619641388581774>, <ServerDescription ('ac-i0nvasr-shard-00-01.ikgqzkz.mongodb.net', 27017) server_type: Unknown, rtt: None>, <ServerDescription ('ac-i0nvasr-shard-00-02.ikgqzkz.mongodb.net', 27017) server_type: RSSecondary, rtt: 0.02401339598408413>]>

In [ ]:
import pandas as pd

df = pd.DataFrame(track_data)
df.set_index("id", inplace=True)
df.info()
df = df[~df["pitches"].isna()]
tolower = lambda s: s.lower()
flatmap = lambda list_of_lists: [item for l in list_of_lists for item in l]
tags = pd.Series(flatmap(df[~df["tags"].isna()]["tags"].values.tolist())).apply(tolower).drop_duplicates()
tags.info()
tags

In [ ]:
pitch_symbol = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]


def pitch_trans(p):
    pitch_frequency = list()
    for timestamp in p:
        pitch_dict = {"timestamp": float(timestamp)}
        for i in range(0, len(p[timestamp])):
            pitch_dict[pitch_symbol[i]] = p[timestamp][i]
        pitch_frequency.append(pitch_dict)
    return pitch_frequency


def pitches_to_dataframe(p):
    return pd.DataFrame(p).sort_values(by=["timestamp"])


pitches = df["pitches"].apply(pitch_trans)
df["pitches"] = pitches
df["pitches"]

In [ ]:
first_song = pitches_to_dataframe(pitches.iloc[0])
first_song

In [ ]:
first_song.plot(x="timestamp", figsize=(40, 10))

In [ ]:
from typing import List

from typing import Dict


def max_of_pitches(freq: List[Dict[str, float]], pitch: chr):
    return max([item[pitch] for item in freq])


def min_of_pitches(freq: List[Dict[str, float]], pitch: chr):
    return min([item[pitch] for item in freq])


for sym in pitch_symbol:
    df.insert(len(df.columns), f"{sym}_max", [max_of_pitches(item, sym) for item in df["pitches"]])
    df.insert(len(df.columns), f"{sym}_min", [min_of_pitches(item, sym) for item in df["pitches"]])
df.info()
temp_df = pd.DataFrame()

for index, row in enumerate(df["pitches"].values):
    song = pitches_to_dataframe(row)
    song["timestamp"] = song["timestamp"].apply(pd.to_timedelta, unit='s')
    resampled: pd.DataFrame = song.set_index("timestamp").resample(
        f"{song.iloc[-1]['timestamp'].total_seconds() * 10 // 1}ms").mean().interpolate()[:100]
    d: pd.DataFrame = pd.DataFrame()
    for col in resampled.columns:
        if col == "timestamp":
            continue
        d = pd.concat([d, pd.DataFrame({f"{col}_{row_idx}": [val] for row_idx, val in enumerate(resampled[col].values)},
                                       index=[df.index[index]])], axis=1)
    temp_df = pd.concat([temp_df, d])

df = pd.concat([df, temp_df], axis=1)
df.info()

In [ ]:
df.drop(columns=["pitches"], inplace=True)
df.to_csv("songs.csv")